## HPGe .mpa data Visualiser

This notebook implements the functions defined in ```hpge_lib.py``` to read .mpa data produced by Ortec HPGe data analysis software. 
It produces an interative plot of the data recorded by one of the four ADC channels availables. 
The code comes with a free-to-use example data file (```test.mpa```).



#### To run the code

* Run cell 1 to initialise the software
* Run cell 2 to load a data file. A file selection window will pop-up.
    * If you want to change the ADC to analyse, run cell 3. Default ADC = 1
    * If you want to plot in log scale, run cell 4. Default = linear scale
* Run cell 5 to plot the data

#### Cell 1: Import libraries and initialise auxiliary variables

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import hpge_lib as hpge
%matplotlib qt

try:
    from tkinter import Tk
    from tkFileDialog import askopenfilenames
except:
    from tkinter import Tk
    from tkinter import filedialog

adc_num = 1
log_scale = False

### Cell 2: Load .mpa data file

In [4]:
root = Tk()
root.withdraw() 
root.wm_attributes('-topmost', 1) 

print('Please, select an .mpa file to analyse')

data_source  =  filedialog.askopenfilenames(parent = root)[0]

print('Done!')

Please, select an .mpa file to analyse
Done!


### Cell 3: Select which ADC to read

In [5]:
adc_num = hpge.configure_ADC()

Which ADC data do I have to analyse? (1,2,3,4; 0 to quit)

Done! Selected ADC:  1


### Cell 4: Select to plot in linear or log scale 

In [6]:
log_scale = hpge.configure_scale()

Do you want the counts to be in log scale? (y/n, 0 to quit): 

Done! Linear scale selected


### Cell 5: Read and plot the data
The calibration is read form the ADC header
* adc_header stores the header of the ADC data
* adc_data contains the counts read by the ADC
* adc_data_errors contains the counts errors (evaluated as the square root of the counts)
* channels contains the channel numbers 
* energy contains the energy of the calibrated channels

In [7]:
datas, adcs_flag_pos, data_flag_pos = hpge.open_data(data_source)
adc_header, adc_data, adc_data_errors = hpge.read_ADC_data(datas, adc_num, adcs_flag_pos, data_flag_pos)

del datas, adcs_flag_pos, data_flag_pos

channels, energies = hpge.calibrate(adc_header)

fig = plt.figure(tight_layout = True, figsize=(10,8))
ax = fig.add_subplot()

ax.errorbar(x=energies, y=adc_data, yerr = adc_data_errors, color = 'black', alpha = 0.5, marker = 'none', fmt='none')
ax.step(energies, adc_data, color = 'black', where = 'mid')

ax.grid(which='both')
ax.set_xlabel('Energy [keV]')
ax.set_ylabel('Counts')

if log_scale:
    ax.set_yscale('log')